In [1]:
from splinter import Browser
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager


executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

In [2]:
url = "https://bhtr.itsyourrace.com/EventCheck.aspx?id=384"
browser.visit(url)
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [3]:
# search for each combination of vowel + consonant

vowels = ['a', 'e', 'i', 'o', 'u']
consonants = ['b', 'c', 'd', 'f', 'g', 'h','j', 'k', 'l', 'm', 'n', 'p', 'q', 'r', 's', 't', 'v', 'w', 'x', 'y', 'z']
combined_list = []

for vowel in vowels:
    for consonant in consonants:
        # combined_list.append(f"{vowel}{consonant}")
        combined_list.append(f"{consonant}{vowel}")

len(combined_list)

105

In [8]:
import re
import pandas as pd
entrants_list = []
hometown_list = []
distance_list = []

def get_entrant():

    try:
        html = browser.html
        soup = BeautifulSoup(html, 'html.parser')

        main_area = soup.find("div", id="ctl00_ContentPlaceHolder1_pnlResults")

        entrants_table = main_area.find('table', class_="regCheck")

        entrants = entrants_table.find_all('tr')

        regex_pattern = r"(\d+\w)"

        for i in range(len(entrants)):

            input_string = str(entrants[i].find('td', class_='col1'))
            matches = re.findall(regex_pattern, input_string)
        
            if matches == ['100M']:
                name = entrants[i].find('td', class_='col2').text.strip()
                location = entrants[i].find('td', class_='col3').text.strip()
                distance = entrants[i].find('td', class_='col1').text.strip()

                if name in entrants_list:
                    next
                
                else:
                    entrants_list.append(name)
                    hometown_list.append(location)
                    distance_list.append(distance)
    except:
        next


In [9]:

for name in combined_list:
    browser.fill('ctl00$ContentPlaceHolder1$txtLastName', name)

    browser.find_by_name('ctl00$ContentPlaceHolder1$btnSearch').first.click()
    
    get_entrant()

In [10]:
entrants_df = pd.DataFrame()

entrants_df['entrant'] = entrants_list
entrants_df['hometown'] = hometown_list
entrants_df['distance'] = distance_list

entrants_df

,entrant,hometown,distance
0,STEVE BAIARDI,"Rowley, MA",100M
1,RUSSELL BARNES,"Oxford, MS",100M
2,ANDREW BARNEY,"American Fork, UT",100M
3,ALEX BARTLEY,"Plainview, MN",100M
4,GARRETT BURBANK,"Powell, WY",100M
...,...,...,...
272,KIRK RUPP,"Tremonton, UT",100M
273,BENJAMIN RUSH,"Clarksville, TN",100M
274,LINDSEY RUST,"Draper, UT",100M
275,JOE SCHRUM,"Hereford, AZ",100M


In [12]:
entrants_df.to_excel('bighorn100_entrants2023.xlsx')